**案例教学3-Agents SDK+Qianwen+MCP智能体开发实战**

1. Agent SDK知识回顾

In [ ]:
!pip install openai==1.99.9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.100.0
    Uninstalling openai-1.100.0:
      Successfully uninstalled openai-1.100.0


In [ ]:
import openai

In [ ]:
openai.__version__

'1.99.9'

In [ ]:
%pip install python-dotenv

In [ ]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 11.8 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel,Agent,Runner,set_default_openai_client
from agents.model_settings import ModelSettings
from contextlib import AsyncExitStack

In [ ]:
%pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)

from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel,Agent,Runner,set_default_openai_client
from agents.model_settings import ModelSettings
from agents import function_tool
import requests,json
import pymysql

from agents import (
    Agent,
    HandoffOutputItem,
    ItemHelpers,
    MessageOutputItem,
    RunContextWrapper,
    Runner,
    ToolCallItem,
    ToolCallOutputItem,
    TResponseInputItem,
    function_tool,
    handoff,
    trace,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

from IPython.display import display, Code, Markdown, Image

In [ ]:
import os
import openai
import glob
import shutil

import numpy as np
import pandas as pd
import pymysql

import json
import io
import inspect
import requests
import re
import random
import string
import base64

from bs4 import BeautifulSoup
import dateutil.parser as parser
import tiktoken
from lxml import etree

import sys
from dotenv import load_dotenv
from openai import OpenAI

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Code, Markdown, Image
from IPython import get_ipython

- Qianwen API调用流程回顾

In [ ]:
# 读取模型API-KEY
api_key = os.getenv("ALIBABA_API_KEY", "sk-XXXXX(自己的API-KEY)")

# 实例化客户端
client = OpenAI(
    api_key=api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 调用 qianwen 模型
response = client.chat.completions.create(

 model="qwen-plus",
 messages=[
     {"role": "system", "content": "You are a helpful coding assistant. Please complete the code implementation."},
     {"role": "user", "content": "您好，请介绍一下你自己"},
 ],
 max_tokens=500,
)

# 输出生成的响应内容
print(response.choices[0].message.content)

您好！我是Qwen，是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我可以帮助您完成很多任务，比如：

- 回答各种问题，包括学术、文化、技术等领域的问题
- 创作文字，如撰写故事、公文、邮件、剧本等
- 进行逻辑推理和编程
- 表达观点，参与讨论
- 玩一些文字游戏

如果您有任何问题或需要帮助，欢迎随时告诉我！


- Agents SDK基本调用流程回顾

In [ ]:
external_client = AsyncOpenAI(
    base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=api_key,
)

set_default_openai_client(external_client)

qianwen_model = OpenAIChatCompletionsModel(
    model="qwen-plus",
    openai_client=external_client)

In [ ]:
agent = Agent(name="Assistant",
              instructions="你是一名助人为乐的助手。",
              model=qianwen_model)

result = await Runner.run(agent, "请写一首关于编程中递归的俳句。")

print(result.final_output)

函数唤自身，  
代码深处无限环，  
递归入梦间。


现在，我们仅需在创建Agent的时候，将MCP服务器视作为一项工具，即可顺利调用MCP服务器进行Agent开发。而实际在借助Agents SDK调用MCP的流程也非常简单，我们`只需将MCP视作tools`，即可进行调用。换而言之，就是如果使用Agents SDK作为Agent开发框架，则可以零门槛快速接入MCP海量服务器生态。

### 2.MCP技术回顾

&emsp;&emsp;MCP，全称是Model Context Protocol，模型上下文协议，由Claude母公司Anthropic于去年11月正式提出。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250318201338022.png" alt="image-20250318201338022" style="zoom:33%;" />

> - Anthropic MCP发布通告：https://www.anthropic.com/news/model-context-protocol
> - MCP GitHub主页：https://github.com/modelcontextprotocol

MCP的核心作用，是统一了Agent开发过程中，大模型调用外部工具的技术实现流程，从而大幅提高Agent开发效率。在MCP诞生之前，不同的外部工具各有不同的调用方法，要连接这些外部工具开发Agent，就必须“每一把锁单独配一把钥匙”，开发工作非常繁琐：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403170211085.png" alt="image-20250403170211085" style="zoom:33%;" />

而MCP的诞生，则统一了这些外部工具的调用流程，使得无论什么样的工具，都可以借助MCP技术按照统一的一个流程快速接入到大模型中，从而大幅加快Agent开发效率。这就好比现在很多设备都可以使用type-c和电脑连接类似。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403170238895.png" alt="image-20250403170238895" style="zoom:33%;" />

从技术实现角度来看，我们可以将MCP看成是Function calling的一种封装，通过server-client架构和一整套开发工具，来规范化Function calling开发流程。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250318202116026.png" alt="image-20250318202116026" style="zoom:33%;" />

换而言之，Agents SDK实现MCP技术流程如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403171429137.png" alt="image-20250403171429137" style="zoom:50%;" />

除此之外，实际上，MCP提供了三个方面的技术支持，其一是抽象的协议，也就是技术理论层面的设计；其二是一整套MCP的开发工具，也就是一些库，方便大家进行MCP工具（MCP Server）的开发并构建智能体；其三，MCP还提供了一整套非常丰富的开发生态，由于MCP是一套标准协议，任何MCP工具（MCP Server）都可以无缝接入任何MCP智能体中，也就是你开发的工具我也能用，我开发的工具你也能直接用。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403170307263.png" alt="image-20250403170307263" style="zoom:33%;" />

### 3.MCP+Agents SDK基础调用流程

&emsp;&emsp;而在新版的Agents SDK中，Agents SDK可以将某个对应的Agent封装为client与外部定义好的server进行通信。基本实现流程如下，

（1） 创建MCP客户端项目

In [ ]:
!pip install uv
!uv init mcp-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 102.5 MB/s eta 0:00:00
Initialized project `mcp-client` at `/content/mcp-client`


In [ ]:
%cd mcp-client

/content/mcp-client


(2） 创建MCP客户端虚拟环境

In [ ]:
# 创建虚拟环境
!uv venv

# 激活虚拟环境
!source .venv/bin/activate

Using CPython 3.12.11 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
? A virtual environment already exists at `.venv`. Do you want to replace it? [y/n] › yes

✔ A virtual environment already exists at `.venv`. Do you want to replace it? · yes
Activate with: source .venv/bin/activate


In [ ]:
!uv add mcp openai openai-agents python-dotenv IPython #为了支持调用大模型，以及在环境变量中读取API-key等信息，需要先安装如下依赖包

Resolved 55 packages in 217ms
Prepared 16 packages in 389ms
Installed 16 packages in 61ms
 + asttokens==3.0.0
 + decorator==5.2.1
 + executing==2.2.0
 + ipython==9.4.0
 + ipython-pygments-lexers==1.1.1
 + jedi==0.19.2
 + matplotlib-inline==0.1.7
 + parso==0.8.4
 + pexpect==4.9.0
 + prompt-toolkit==3.0.51
 + ptyprocess==0.7.0
 + pure-eval==0.2.3
 + pygments==2.19.2
 + stack-data==0.6.3
 + traitlets==5.14.3
 + wcwidth==0.2.13


In [ ]:
# 创建 .env 文件
!echo "OPENAI_API_KEY=sk-XXXXX(自己的API-KEY)" > /content/mcp-client/.env
!echo "BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1" >> /content/mcp-client/.env
!echo "MODEL=qwen-plus" >> /content/mcp-client/.env

创建名称为`weather_server.py`的天气服务

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403173507052.png" alt="image-20250403173507052" style="zoom:33%;" />

代码如下：

In [ ]:
import os
import json
import httpx
from typing import Any
from dotenv import load_dotenv
from mcp.server.fastmcp import FastMCP

# 初始化 MCP 服务器
mcp = FastMCP("WeatherServer")
# OpenWeather API 配置
OPENWEATHER_API_BASE = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "XXXXX(自己的API-KEY)" # 填写你的OpenWeather-API-KEY
USER_AGENT = "weather-app/1.0"

async def fetch_weather(city: str) -> dict[str, Any] | None:
  """从 OpenWeather API 获取天气信息。
  :param city: 城市名称（需使用英文，如 Beijing）
  :return: 天气数据字典；若出错返回包含 error 信息的字典
  """
  params = {
    "q": city,
    "appid": API_KEY,
    "units": "metric",
    "lang": "zh_cn"
  }
  headers = {"User-Agent": USER_AGENT}


  async with httpx.AsyncClient() as client:
    try:
      response = await client.get(OPENWEATHER_API_BASE, params=params,
      headers=headers, timeout=30.0)
      response.raise_for_status()
      return response.json() # 返回字典类型
    except httpx.HTTPStatusError as e:
      return {"error": f"HTTP 错误: {e.response.status_code}"}
    except Exception as e:
      return {"error": f"请求失败: {str(e)}"}


def format_weather(data: dict[str, Any] | str) -> str:
  """
  将天气数据格式化为易读文本。
  :param data: 天气数据（可以是字典或 JSON 字符串）
  :return: 格式化后的天气信息字符串
  """
  # 如果传入的是字符串，则先转换为字典
  if isinstance(data, str):
    try:
      data = json.loads(data)
    except Exception as e:
      return f"无法解析天气数据: {e}"
  # 如果数据中包含错误信息，直接返回错误提示
  if "error" in data:
    return f"⚠️ {data['error']}"
  # 提取数据时做容错处理
  city = data.get("name", "未知")
  country = data.get("sys", {}).get("country", "未知")
  temp = data.get("main", {}).get("temp", "N/A")
  humidity = data.get("main", {}).get("humidity", "N/A")
  wind_speed = data.get("wind", {}).get("speed", "N/A")
  # weather 可能为空列表，因此用 [0] 前先提供默认字典
  weather_list = data.get("weather", [{}])
  description = weather_list[0].get("description", "未知")
  return (
    f"🌍 {city}, {country}\n"
    f"🌡 温度: {temp}°C\n"
    f"💧 湿度: {humidity}%\n"
    f"🌬 风速: {wind_speed} m/s\n"
    f"🌤 天气: {description}\n"
  )



@mcp.tool()
async def query_weather(city: str) -> str:
  """
  输入指定城市的英文名称，返回今日天气查询结果。
  :param city: 城市名称（需使用英文）
  :return: 格式化后的天气信息
  """
  data = await fetch_weather(city)
  return format_weather(data)


if __name__ == "__main__":
  # 以标准 I/O 方式运行 MCP 服务器
  mcp.run(transport='stdio')

创建名称为client.py的客户端，然后导入代码如下：

In [ ]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)
import asyncio
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel,Agent,Runner,set_default_openai_client
from agents.model_settings import ModelSettings
from agents import function_tool
import requests,json

from agents import (
    Agent,
    HandoffOutputItem,
    ItemHelpers,
    MessageOutputItem,
    RunContextWrapper,
    Runner,
    ToolCallItem,
    ToolCallOutputItem,
    TResponseInputItem,
    function_tool,
    handoff,
    trace,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents.mcp import MCPServer, MCPServerStdio



openai_api_key = os.getenv("OPENAI_API_KEY") # 读取 OpenAI API Key
base_url = os.getenv("BASE_URL") # 读取 BASE YRL
model = os.getenv("MODEL") # 读取 model
if not openai_api_key:
  raise ValueError("❌ 未找到 OpenAI API Key，请在 .env 文件中设置OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key, base_url=base_url)

external_client = AsyncOpenAI(
    base_url = base_url,
    api_key=openai_api_key,
)

set_default_openai_client(external_client)

qianwen_model = OpenAIChatCompletionsModel(
    model=model,
    openai_client=external_client)

async def run(mcp_server: MCPServer):

    agent = Agent(
        name="Assistant",
        instructions="你是一名助人为乐的助手",
        mcp_servers=[mcp_server],
        model=qianwen_model
    )

    message = "请帮我查询北京今天天气如何？"
    print(f"Running: {message}")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)

async def mcp_run():
    async with MCPServerStdio(
        name="Weather Server",
        cache_tools_list=True,
        params = {"command": "uv","args": ["run", "weather_server.py"]}
    ) as server:
        await run(server)

async def main():
   await mcp_run()

if __name__ == "__main__":
  asyncio.run(main())


具体解释如下：同时定义Agent+MCP运行函数，要求带入MCPServer对象，且带入mcp_servers中，作为类似tools参数带入到当前Agent运行过程中：

In [ ]:
async def run(mcp_server: MCPServer):

    agent = Agent(
        name="Assistant",
        instructions="你是一名助人为乐的助手",
        mcp_servers=[mcp_server],
        model=qianwen_model
    )

    message = "请帮我查询北京今天天气如何？"
    print(f"Running: {message}")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)

然后创建mcp_run函数，负责开启外部server并运行Agent：

In [ ]:
async def mcp_run():
    async with MCPServerStdio(
        name="Weather Server",
        cache_tools_list=True,
        params = {"command": "uv","args": ["run", "weather_server.py"]}
    ) as server:
        await run(server)

✅ 关键组件解释：

- `async with MCPServerStdio(...) as server:`  
  启动一个 MCP 工具服务器进程，使用标准输入输出（`stdio`）作为通信协议，并在上下文中运行（退出时会自动关闭）。

- `name="Weather Server"`  
  给这个 MCP Server 起名为“天气服务器”，这只是用于日志和识别用的标识符。

- `cache_tools_list=True`  
  意思是：首次加载工具时缓存工具列表，后续不需要重新请求工具元数据（提升效率）。

- `params = {"command": "uv", "args": ["run", "weather_server.py"]}`  
  这是启动 MCP 工具服务器的 **命令行参数**，等价于在命令行里运行：
  ```bash
  uv run weather_server.py
  ```
  - `uv`：同样使用[uv](https://github.com/astral-sh/uv)，是一个快速的 Python 包和环境管理器，比 pip/venv 更高效。
  - `run weather_server.py`：运行一个你定义好的MCP server。


最后测试运行：

In [ ]:
!uv pip install openai==1.99.9
!uv pip install openai-agents
!uv pip install mcp
!uv pip install httpx
!uv pip install pydantic
!uv pip install python-dotenv
!uv pip install agents
!uv pip install mcp-agents

Using Python 3.12.11 environment at: /usr
Audited 1 package in 127ms
Using Python 3.12.11 environment at: /usr
Audited 1 package in 118ms
Using Python 3.12.11 environment at: /usr
Audited 1 package in 84ms
Using Python 3.12.11 environment at: /usr
Audited 1 package in 82ms
Using Python 3.12.11 environment at: /usr
Audited 1 package in 97ms
Using Python 3.12.11 environment at: /usr
Audited 1 package in 99ms
Using Python 3.12.11 environment at: /usr
Audited 1 package in 92ms
Using Python 3.12.11 environment at: /usr
Audited 1 package in 110ms


In [ ]:
!uv run client.py

Running: 请帮我查询北京今天天气如何？
Processing request of type ListToolsRequest
Processing request of type CallToolRequest
HTTP Request: GET https://api.openweathermap.org/data/2.5/weather?q=Beijing&appid=11e9adb827a70b8bbdee0f3e31270429&units=metric&lang=zh_cn "HTTP/1.1 200 OK"
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-21247***********************92e9. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
北京今天的天气情况如下：

- **温度**: 24.94°C  
- **湿度**: 62%  
- **风速**: 3.85 m/s  
- **天气**: 阴，多云  

希望这些信息对你有帮助！
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-21247***********************92e9. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}


### 4. Agents SDK接入更多开源MCP服务器流程

&emsp;&emsp;若要采用MCP技术栈，最核心的便利就在于可以快速接入海量MCP开源服务器，无需反复开发，即可快速丰富当前Agent功能。热门MCP server合集地址如下：

- Model Context Protocol servers: https://github.com/modelcontextprotocol/servers
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403174645233.png" alt="image-20250403174645233" style="zoom:33%;" />

- Awesome MCP Servers: https://github.com/punkpeye/awesome-mcp-servers
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403174514932.png" alt="image-20250403174514932" style="zoom:33%;" />

- MCP导航： https://mcp.so/
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403174757712.png" alt="image-20250403174757712" style="zoom:50%;" /></center>

接下来我们就先尝试接入一个开源的、开发人员常用的MCP服务器——mcp-server-git。

- mcp-server-git：https://github.com/modelcontextprotocol/servers/tree/main/src/git
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403175004931.png" alt="image-20250403175004931" style="zoom:33%;" />

In [ ]:
!uvx git+https://github.com/modelcontextprotocol/servers/tree/main/src/git

  × Failed to resolve `--with` requirement
  ╰─▶ Git operation failed


##### 📌 MCP-Server-Git 简介

`mcp-server-git` 是一个遵循 **Model Context Protocol (MCP)** 的 Git 操作服务器，专为大语言模型与 Git 仓库的交互而设计。通过该服务，模型可以安全、结构化地完成 Git 操作，包括状态查询、差异比较、提交更改、分支管理等，从而实现自动化代码管理与协作。


✨ 核心功能包括：

- **查询仓库状态**：获取当前工作区和暂存区的变动情况（`git_status`、`git_diff_unstaged`、`git_diff_staged`）
- **版本比较**：支持分支或提交之间的差异查看（`git_diff`）
- **代码提交与暂存管理**：支持新增、暂存、撤销暂存、更改提交（`git_add`、`git_reset`、`git_commit`）
- **日志查询与历史查看**：获取提交历史、查看具体提交内容（`git_log`、`git_show`）
- **分支操作**：新建分支、切换分支（`git_create_branch`、`git_checkout`）
- **仓库初始化**：支持新建空 Git 仓库（`git_init`）

##### 🚀 调用方式
✅ 使用 `uvenv` 快速启动（推荐方式）

无需安装，只需一行命令即可运行：

```bash
uvenv run mcp-server-git
```

首次运行会从 PyPI 下载并缓存，后续启动速度更快。


📡 接口调用格式（示例）

以 `git_status` 为例，MCP 工具调用格式如下：

```json
{
  "tool": "git_status",
  "input": {
    "repo_path": "/path/to/your/git/repo"
  }
}
```

服务将返回 Git 工作目录当前状态的文本描述。

- uvx（已弃用）/ uvenv 工具安装与使用指南（Ubuntu）

`uvx`（现已由 `uvenv` 取代）是一个基于 Python 的零安装运行工具，可快速运行 Python 包，而无需手动安装或污染系统环境。

1. uvenv 安装方法（Ubuntu）

打开终端，运行以下命令自动安装最新版 `uv`（含 `uvenv`）：

```bash
curl -Ls https://astral.sh/uv/install.sh | sh
```

或使用 `wget` 命令安装：

```bash
wget -qO- https://astral.sh/uv/install.sh | sh
```

安装完成后，验证是否安装成功：

```bash
uv --version
uvenv --help
```

🔁 可选：通过 pip 安装（不推荐）

```bash
pip install uv
```

虽然可以使用 pip 安装 `uv`，但官方推荐使用上面的安装脚本，因为：
- 安装的是预编译二进制版本，性能更好
- 功能更完整
- 自动将可执行文件添加到环境路径


2. uvenv 功能说明

- **无需提前安装**：`uvenv` 自动从 PyPI 拉取并运行 Python 包。
- **隔离环境**：每次执行均为临时环境，避免污染系统全局环境。
- **缓存机制**：首次拉取后自动缓存，后续使用直接读取缓存，提高运行速度。
- **用途广泛**：适用于快速测试、命令行工具临时使用以及 Python 脚本的隔离执行。


3. uvenv 基本使用方法

使用语法

```bash
uvenv run <package-name>
```

示例：运行 `mcp-server-git`

首次运行会自动从 PyPI 下载并缓存：

```bash
uvenv run mcp-server-git
```

再次运行则直接使用缓存，速度更快：

```bash
uvenv run mcp-server-git
```

⚠️ 更新缓存中的包

默认情况下，`uvenv` 不会自动检测和使用 PyPI 上的新版本。如果想强制使用最新版本，可以：

- 清理缓存后重新运行：

```bash
uv cache clear
uvenv run mcp-server-git
```


4. 常用操作

- **查看缓存位置**：

```bash
ls ~/.cache/uv
```

- **清理缓存**（若需要）：

```bash
uv cache clear
```

- **升级 uv/uvenv 工具**：

```bash
uv self update
```

- **从 uvx 迁移提示**：如果你看到如下提示：

```bash
Deprecation Message: `uvx` is deprecated in favor of `uvenv`.
```
说明你已成功迁移到新版，请使用 `uvenv run <package>` 来代替原来的 `uvx <package>` 命令。

In [ ]:
!pip install uv

In [ ]:
!wget -qO- https://astral.sh/uv/install.sh | sh

downloading uv 0.8.13 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [ ]:
!uv --version
!uvx --help

uv 0.8.13
Run a command provided by a Python package.

Usage: uvx [OPTIONS] [COMMAND]

Options:
      --from <FROM>
          Use the given package to provide the command
  -w, --with <WITH>
          Run with the given packages installed
      --with-editable <WITH_EDITABLE>
          Run with the given packages installed in editable mode
      --with-requirements <WITH_REQUIREMENTS>
          Run with all packages listed in the given `requirements.txt` files
  -c, --constraints <CONSTRAINTS>
          Constrain versions using the given requirements files [env: UV_CONSTRAINT=]
  -b, --build-constraints <BUILD_CONSTRAINTS>
          Constrain build dependencies using the given requirements files when building source
          distributions [env: UV_BUILD_CONSTRAINT=]
      --overrides <OVERRIDES>
          Override versions using the given requirements files [env: UV_OVERRIDE=]
      --isolated
          Run the tool in an isolated virtual environment, ignoring any already-installed to

In [ ]:
!git clone https://github.com/modelcontextprotocol/servers.git

Cloning into 'servers'...
remote: Enumerating objects: 12176, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 12176 (delta 201), reused 136 (delta 135), pack-reused 11946 (from 3)
Receiving objects: 100% (12176/12176), 25.71 MiB | 36.72 MiB/s, done.
Resolving deltas: 100% (7838/7838), done.


In [ ]:

!uv --directory /content/servers/src/git run mcp-server-git



Aborted!


创建脚本.py

```python
import asyncio
import shutil

from agents import Agent, Runner, trace
from agents.mcp import MCPServer, MCPServerStdio


async def run(mcp_server: MCPServer, directory_path: str):
    agent = Agent(
        name="Assistant",
        instructions=f"Answer questions about the git repository at {directory_path}, use that for repo_path",
        mcp_servers=[mcp_server],
        model=deepseek_model
    )

    message = "请帮我介绍下这个项目。"
    print("\n" + "-" * 40)
    print(f"Running: {message}")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)


async def main():
    async with MCPServerStdio(
        cache_tools_list=True,
        params={"command": "uvenv", "args": ["run", "mcp-server-git"]},
    ) as server:
        await run(server, directory_path)


if __name__ == "__main__":
    if not shutil.which("uvenv"):
        raise RuntimeError("uvx is not installed. Please install it with `pip install uvx`.")

    asyncio.run(main())
```

### (2). Agents SDK接入多个MCP服务器流程

&emsp;&emsp;接下来我们继续介绍如何将Agents SDK同时接入多个MCP服务器，理论上，MCP一个服务器能同时运行多个外部函数，而一个MCP Client则可以连接多个MCP服务器：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250403183654681.png" alt="image-20250403183654681" style="zoom:33%;" />

而Agents SDK本身也是可以作为MCP Client的，因此是完全可以连接多个MCP server，其基本实现函数如下：

In [ ]:
async def mcp_run_multi(servers_params, message):
    # 使用 AsyncExitStack 自动管理多个上下文退出
    async with AsyncExitStack() as stack:
        servers = []
        # 创建并进入所有 server 上下文
        for p in servers_params:
            server = MCPServerStdio(
                name=p.get("name", "Unnamed Server"),
                cache_tools_list=True,
                params={
                    "command": "uv",
                    "args": ["run", p["script"]],
                },
            )
            entered_server = await stack.enter_async_context(server)
            servers.append(entered_server)

        # 构造 agent，传入多个 server
        agent = Agent(
            name="Assistant",
            instructions="你是一名助人为乐的助手",
            mcp_servers=servers,
            model_settings=ModelSettings(tool_choice="required"),
            model=deepseek_model
        )

        print(f"Running: {message}")
        result = await Runner.run(starting_agent=agent, input=message)
        print(result.final_output)

        return result

这里我们尝试创建一个“写入本地文档”的MCP服务器：`write_server.py`，代码如下：

```python
import json
import httpx
from typing import Any
from mcp.server.fastmcp import FastMCP

# 初始化 MCP 服务器
mcp = FastMCP("WriteServer")
USER_AGENT = "write-app/1.0"

@mcp.tool()
async def write_file(content: str) -> str:
    """
    将指定内容写入本地文件。
    :param content: 必要参数，字符串类型，用于表示需要写入文档的具体内容。
    :return：是否成功写入
    """
    return "已成功写入本地文件。"

if __name__ == "__main__":
    # 以标准 I/O 方式运行 MCP 服务器
    mcp.run(transport='stdio')
```

In [ ]:
!uv run client1.py

然后尝试同时调用多个server，结果如下：

In [ ]:
# 示例调用：传入多个 server 的配置
result = await mcp_run_multi(
    servers_params=[
        {"name": "Weather Server", "script": "weather_server.py"},
        {"name": "Writer Server", "script": "write_server.py"}
    ],
    message="请帮我查询北京天气，并写入本地文档。"
)

Running: 请帮我查询北京天气，并写入本地文档。
北京的天气信息已查询并成功写入本地文档：

🌍 Beijing, CN  
🌡 温度: 19.94°C  
💧 湿度: 13%  
🌬 风速: 5.51 m/s  
🌤 天气: 多云  


In [ ]:
len(result.new_items)

5

In [ ]:
result.new_items[0].raw_item

ResponseFunctionToolCall(arguments='{"city":"Beijing"}', call_id='call_0_a4e6781f-0fcd-46e6-ac92-b0d3b362a83f', name='query_weather', type='function_call', id='__fake_id__', status=None)

In [ ]:
result.new_items[1].raw_item

{'call_id': 'call_0_a4e6781f-0fcd-46e6-ac92-b0d3b362a83f',
 'output': '{"type":"text","text":"🌍 Beijing, CN\\n🌡 温度: 19.94°C\\n💧 湿度: 13%\\n🌬 风速: 5.51 m/s\\n🌤 天气: 多云\\n","annotations":null}',
 'type': 'function_call_output'}

In [ ]:
result.new_items[2].raw_item

ResponseFunctionToolCall(arguments='{"content":"🌍 Beijing, CN\\n🌡 温度: 19.94°C\\n💧 湿度: 13%\\n🌬 风速: 5.51 m/s\\n🌤 天气: 多云"}', call_id='call_0_e137b744-d34a-4dba-8758-456c2aca5eb0', name='write_file', type='function_call', id='__fake_id__', status=None)

In [ ]:
result.new_items[3].raw_item

{'call_id': 'call_0_e137b744-d34a-4dba-8758-456c2aca5eb0',
 'output': '{"type":"text","text":"已成功写入本地文件。","annotations":null}',
 'type': 'function_call_output'}

In [ ]:
result.new_items[4].raw_item

ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='北京的天气信息已查询并成功写入本地文档：\n\n🌍 Beijing, CN  \n🌡 温度: 19.94°C  \n💧 湿度: 13%  \n🌬 风速: 5.51 m/s  \n🌤 天气: 多云  ', type='output_text')], role='assistant', status='completed', type='message')

In [ ]:
result.to_input_list()

[{'content': '请帮我查询北京天气，并写入本地文档。', 'role': 'user'},
 {'arguments': '{"city":"Beijing"}',
  'call_id': 'call_0_a4e6781f-0fcd-46e6-ac92-b0d3b362a83f',
  'name': 'query_weather',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'call_0_a4e6781f-0fcd-46e6-ac92-b0d3b362a83f',
  'output': '{"type":"text","text":"🌍 Beijing, CN\\n🌡 温度: 19.94°C\\n💧 湿度: 13%\\n🌬 风速: 5.51 m/s\\n🌤 天气: 多云\\n","annotations":null}',
  'type': 'function_call_output'},
 {'arguments': '{"content":"🌍 Beijing, CN\\n🌡 温度: 19.94°C\\n💧 湿度: 13%\\n🌬 风速: 5.51 m/s\\n🌤 天气: 多云"}',
  'call_id': 'call_0_e137b744-d34a-4dba-8758-456c2aca5eb0',
  'name': 'write_file',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'call_0_e137b744-d34a-4dba-8758-456c2aca5eb0',
  'output': '{"type":"text","text":"已成功写入本地文件。","annotations":null}',
  'type': 'function_call_output'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': '北京的天气信息已查询并成功写入本地文档：\n\n🌍 Beijing, CN  \n🌡 温度: 19.94°C  \n💧 湿度: 13%  \n🌬 风

不难看出，Agents SDK对于MCP实现过程基本遵照Function calling来执行。

In [ ]:
# 示例调用：传入多个 server 的配置
result = await mcp_run_multi(
    servers_params=[
        {"name": "Weather Server", "script": "weather_server.py"},
        {"name": "Writer Server", "script": "write_server.py"}
    ],
    message="请帮我查询北京和杭州天气，并写入本地文档。"
)

Running: 请帮我查询北京和杭州天气，并写入本地文档。
已成功查询北京和杭州的天气信息，并写入本地文档。以下是天气详情：

### 北京天气
- **温度**: 17.94°C
- **湿度**: 16%
- **风速**: 3.24 m/s
- **天气**: 多云

### 杭州天气
- **温度**: 14.95°C
- **湿度**: 40%
- **风速**: 3.22 m/s
- **天气**: 晴

文件已保存。


In [ ]:
len(result.new_items)

7

In [ ]:
result.to_input_list()

[{'content': '请帮我查询北京和杭州天气，并写入本地文档。', 'role': 'user'},
 {'arguments': '{"city": "Beijing"}',
  'call_id': 'call_0_fe02f63b-7a08-4983-8098-6e50821100ac',
  'name': 'query_weather',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'arguments': '{"city": "Hangzhou"}',
  'call_id': 'call_1_98d66c45-6268-42b5-bc0b-15a4cd857ece',
  'name': 'query_weather',
  'type': 'function_call',
  'id': '__fake_id__'},
 {'call_id': 'call_0_fe02f63b-7a08-4983-8098-6e50821100ac',
  'output': '{"type":"text","text":"🌍 Beijing, CN\\n🌡 温度: 17.94°C\\n💧 湿度: 16%\\n🌬 风速: 3.24 m/s\\n🌤 天气: 多云\\n","annotations":null}',
  'type': 'function_call_output'},
 {'call_id': 'call_1_98d66c45-6268-42b5-bc0b-15a4cd857ece',
  'output': '{"type":"text","text":"🌍 Hangzhou, CN\\n🌡 温度: 14.95°C\\n💧 湿度: 40%\\n🌬 风速: 3.22 m/s\\n🌤 天气: 晴\\n","annotations":null}',
  'type': 'function_call_output'},
 {'arguments': '{"content":"🌍 Beijing, CN\\n🌡 温度: 17.94°C\\n💧 湿度: 16%\\n🌬 风速: 3.24 m/s\\n🌤 天气: 多云\\n\\n🌍 Hangzhou, CN\\n🌡 温度: 14.95°C\\n💧 

整体实现流程和上一小节介绍的Agents SDK Function calling执行流程完全一致：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250401162048526.png" alt="image-20250401162048526" style="zoom:50%;" />